# <font color='blue'>Data Science Academy - Python Fundamentos - Capítulo 6</font>

## Download: http://github.com/dsacademybr

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

## Preparando a Conexão com o Twitter

In [1]:
# Instala o pacote tweepy
!pip install tweepy

tensorflow 2.1.0 requires absl-py>=0.7.0, which is not installed.
tensorflow 2.1.0 requires astor>=0.6.0, which is not installed.
tensorflow 2.1.0 requires gast==0.2.2, which is not installed.
tensorflow 2.1.0 requires google-pasta>=0.1.6, which is not installed.
tensorflow 2.1.0 requires grpcio>=1.8.6, which is not installed.
tensorflow 2.1.0 requires keras-applications>=1.0.8, which is not installed.
tensorflow 2.1.0 requires keras-preprocessing>=1.1.0, which is not installed.
tensorflow 2.1.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.1.0 requires protobuf>=3.8.0, which is not installed.
tensorflow 2.1.0 requires tensorflow-estimator<2.2.0,>=2.1.0rc0, which is not installed.
tensorflow 2.1.0 requires termcolor>=1.1.0, which is not installed.
tensorboard 2.1.0 requires absl-py>=0.4, which is not installed.
tensorboard 2.1.0 requires google-auth-oauthlib<0.5,>=0.4.1, which is not installed.
tensorboard 2.1.0 requires grpcio>=1.24.3, which is not installed.
tensor

In [2]:
# Importando os módulos Tweepy, Datetime e Json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

Veja no manual em pdf como criar sua API no Twitter e configure as suas chaves abaixo.

In [ ]:
api_key="Hxh3XPIdMuvGi6CRhaJplZVQF"
api_secret= "ZHrw0QbqqBElOtdWbZK90s6QYTl6BUj24Aad0462W3KHTpZVAh"
access_token = "846412731926413317-GHRx2c3ANGqJkgBJIb9YJYhnQ4adBY3"
access_token_secret= "cke8pH4s2Eiexw1ln3N71dCbiAnhZO8mrDaddpVO0a94v"

In [3]:
# Adicione aqui sua Consumer Key
consumer_key = "Hxh3XPIdMuvGi6CRhaJplZVQF"

In [4]:
# Adicione aqui sua Consumer Secret 
consumer_secret = "ZHrw0QbqqBElOtdWbZK90s6QYTl6BUj24Aad0462W3KHTpZVAh"

In [5]:
# Adicione aqui seu Access Token
access_token = "846412731926413317-GHRx2c3ANGqJkgBJIb9YJYhnQ4adBY3"

In [6]:
# Adicione aqui seu Access Token Secret
access_token_secret = "cke8pH4s2Eiexw1ln3N71dCbiAnhZO8mrDaddpVO0a94v"

In [7]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [8]:
auth.set_access_token(access_token, access_token_secret)

In [8]:
# Criando uma classe para capturar os stream de dados do Twitter e 
# armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at":created_at,"id_str":id_str,"text":text,}
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

In [9]:
# Criando o objeto mylistener
mylistener = MyListener()

In [18]:
# Criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

## Preparando a Conexão com o MongoDB

In [19]:
# Importando do PyMongo o módulo MongoClient
from pymongo import MongoClient   

In [20]:
# Criando a conexão ao MongoDB
client = MongoClient('localhost', 27017)

In [21]:
# Criando o banco de dados twitterdb
db = client.twitterdb

In [22]:
# Criando a collection "col"
col = db.tweets 

In [23]:
# Criando uma lista de palavras chave para buscar nos Tweets
keywords = ['economia', 'covid-19']

## Coletando os Tweets

In [24]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

## --> Pressione o botão Stop na barra de ferramentas para encerrar a captura dos Tweets

## Consultando os Dados no MongoDB

In [25]:
mystream.disconnect()

In [26]:
# Verificando um documento no collection
col.find_one()

{'_id': ObjectId('5eb295afdda712d79ae78f05'),
 'created_at': 'Wed May 06 10:47:06 +0000 2020',
 'id_str': '1257985235075174400',
 'text': 'RT @dagenmcdowell: In late March, NY *ordered* nursing homes to accept Covid-19 patients discharged from hospitals. When asked about this d…'}

## Análise de Dados com Pandas e Scikit-Learn

In [27]:
# criando um dataset com dados retornados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [28]:
# Importando o módulo Pandas para trabalhar com datasets em Python
import pandas as pd

In [29]:
# Criando um dataframe a partir do dataset 
df = pd.DataFrame(dataset)

In [42]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import string
from wordcloud import WordCloud
import re

In [36]:
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wesley.neves\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
custon_stop_words = ['nessa','todo','pra','vou','aqui','se','.',',','lá','vcs','pois','"',':','``']
punctuation = list(string.punctuation)
lista_stopwords  = set(stopwords.words('portuguese') + punctuation + custon_stop_words+ ['rt', 'via'])


In [43]:
def remove_url(texto):
    return re.sub(r'http\S+', '', texto)


def set_tokenize(texto):
    return word_tokenize(texto)


def remove_stop_Words(texto,lista_stopwords):
    palavras =set_tokenize(texto)
    palavras_sem_stopwords = [palavra.lower() for palavra in palavras if palavra.lower() not in lista_stopwords]
    return (palavras_sem_stopwords)

def remove_caracteres_especiais(texto):
    punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
    remove = ''.join([i for i in texto if not i in punctuations])
    remove_1 =  re.sub(r" ?\([^)]+\)", "", remove)
    remove_2 =''.join(remove_1).replace("'", " ")
    remove_3 =''.join(remove_2).replace("[", " ")
    remove_4 =''.join(remove_3).replace("]", " ")
    return str(remove_4)   


### recuperar o conteudo a ser trabalhado
def normaliza_dados(texto,lista_stopwords):
    valor = remove_url(texto)
    valor_2 = remove_caracteres_especiais(valor)
    remove =remove_stop_Words(valor_2,lista_stopwords)

    return remove 

In [57]:
# Imprimindo o dataframe
df["TextoLimpo"] =""

In [58]:
df['TextoLimpo'] = [" ".join(normaliza_dados(texto,lista_stopwords)) for texto in df['text']]

In [60]:
df["TextoLimpo"][0:5]

0    dagenmcdowell in late march ny ordered nursing...
1    drkoko28 sudah diumumkan kemarin berikut alur ...
2    ani delhi police constable whose covid19 test ...
3    bjp4india covid19 की वजह से विदेशों में फंसे भ...
4    drmahreenbhutto get well soon rashid bhai sind...
Name: TextoLimpo, dtype: object

In [61]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.TextoLimpo)

In [62]:
df.head(2)

,created_at,text,TextoLimpo,ConteudoLimpo
0,Wed May 06 10:47:06 +0000 2020,"RT @dagenmcdowell: In late March, NY *ordered*...",dagenmcdowell in late march ny ordered nursing...,dagenmcdowell in late march ny ordered nursing...
1,Wed May 06 10:47:07 +0000 2020,RT @dr_koko28: Sudah diumumkan kemarin berikut...,drkoko28 sudah diumumkan kemarin berikut alur ...,drkoko28 sudah diumumkan kemarin berikut alur ...


In [63]:
# Importando o módulo Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer

In [64]:
# Contando o número de ocorrências das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,covid19,2191
1,the,1416
2,of,826
3,to,761
4,in,727
5,and,532
6,la,427
7,is,329
8,on,321
9,el,303


# Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>